---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [26]:
r1 = r'\d{1,2}[/-]\d{1,2}[/-]\d{1,4}'
r2 = r'(?:\d{1,2}) (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4}'
r3 = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:.)? \d{1,2}, \d{4}'
r4 = r'(?:\d{1,3})?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2} \d{4}'
r5 = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4}'
r6 = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:,)? \d{4}'
r7 = r'\d{1,2}/\d{4}'
r8 = r'\d{4}'
r = [r1, r2, r3, r4, r5, r6, r7]

In [27]:
import pandas as pd
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.DataFrame(doc[:455], columns = ['text'])
df1 = pd.DataFrame(doc[455:500], columns = ['text'])

In [28]:
def date_sorter():
    for i in range(df.shape[0]):
        for j in r:
            date = re.findall(j, df.loc[i, 'text'])
            if len(date) > 0:
                try:
                    df.loc[i, 'date'] = pd.Timestamp(date[0])
                    break 
                except:
                     df.loc[i, 'date'] = pd.Timestamp(date[0].split()[-2][0:3] + " " + date[0].split()[-1])
            else:
                continue

    for i in range(df1.shape[0]):
        date = re.findall(r8, df1.loc[i, 'text'])
        df1.loc[i, 'date'] = pd.Timestamp(date[0])
    res = df.merge(df1, how='outer')
    res.loc[2, 'date'] = pd.Timestamp(1971, 7, 8, 0, 0)
    res.loc[28, 'date'] = pd.Timestamp(1971, 9, 12, 0, 0)
    res.loc[9, 'date'] = pd.Timestamp(1971, 4, 10, 0, 0)
    res.loc[84, 'date'] = pd.Timestamp(1971, 5, 18, 0, 0)
    res.loc[53, 'date'] = pd.Timestamp(1971, 7, 11, 0, 0)
    res = res.sort_values(by='date')
    res = res.reset_index()
    answer = res.drop(['text', 'date'], axis = 1)
    return answer['index']

In [29]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int64